In [1]:
import pandas as pd
import torch
import secret
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [2]:
secret

<module 'secret' from 'c:\\Users\\ankaz\\OneDrive\\Dokumenty\\GitHUB\\Bielik_projects\\bielik_projects\\secret.py'>

In [3]:
def load_dataset():  # -> pd.DataFrame:
    """
    Wczytuje dane z arkusza Google Sheets, usuwa nadmiarowe kolumny i filtruje wiersze z "prawidłowe" == "T".

    Returns:
        pd.DataFrame: Odfiltrowana ramka danych.
    """
    # URL do arkusza Google Sheets w formacie CSV
    sheet_url = "https://docs.google.com/spreadsheets/d/1-Ag5DOHUywOeg_lwDi8ymRsREmHZ87s6d6FUPfLr8tc/export?format=csv"
    
    # Wczytanie arkusza
    df = pd.read_csv(sheet_url)

    # Usunięcie nadmiarowych kolumn (np. 'Unnamed')
    df.drop(columns=df.columns[df.columns.str.contains('^Unnamed')], inplace=True)

    # Filtrowanie wierszy, gdzie "Prawidłowe" == "T"
    if "Prawidłowe" in df.columns:
        df = df.loc[df["Prawidłowe"] == "T"]
    else:
        raise ValueError("Brak wymaganej kolumny 'prawidłowe' w ramce danych.")
    
    return df

In [4]:
df = load_dataset()
df.head()

,Pytanie,Odpowiedź,link do testu w B2,rodzaj zagadki,Zaskakujące odpowiedzi Bielika,Prawidłowe
0,Kto ma imieniny 30 lutego?,"Nikt, bo nie ma takiego dnia.",NaN,NaN,NaN,T
1,Ile przekątnych ma trójkąt?,"Zero, trójkąt nie ma przekątnych",NaN,sprzeczność z uwagi na definicję pojęcia,NaN,T
2,W którym roku wydarzyła się na terenie Polski ...,Nie było takiej bitwy.,NaN,NaN,NaN,T
3,W którym miesiącu zaczyna się 5 kwartał roku?,Rok ma tylko 4 kwartały.,NaN,NaN,NaN,T
4,Ile skrzydeł ma pies?,Zero. Psy nie mają skrzydeł.,NaN,NaN,NaN,T


In [5]:
# wczytanie modelu 
model_id = "speakleash/Bielik-7B-Instruct-v0.1"

# kwantyzacja modelu
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=quantization_config
)

# Wczytanie tokenizer z przypisaniem pad_token_id
tokenizer = AutoTokenizer.from_pretrained(model_id)
# tokenizer.pad_token = tokenizer.eos_token  # Ustawienie pad_token_id jako eos_token_id

pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,
    return_full_text=False
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


In [6]:
def get_answer(df, pipe, model_id, num_rows = None):
    """
    Generuje odpowiedzi dla pytań w kolumnie 'Pytanie' i zapisuje je w nowej kolumnie 
    z nazwą 'Odpowiedź: model_id'.

    Args:
        df (pd.DataFrame): Ramka danych z kolumną 'Pytanie'.
        pipe: Pipeline modelu językowego.
        model_id (str): Nazwa modelu, która zostanie dodana do nagłówka kolumny odpowiedzi.
        num_rows (int, optional): Liczba wierszy do przetworzenia. Przetwarza wszystkie wiersze, jeśli None.

    Returns:
        pd.DataFrame: Zaktualizowana ramka danych z nową kolumną 'Odpowiedź: model_id'.
    """
    # Jeśli num_rows jest None, przetwarzaj wszystkie wiersze
    df_subset = df if num_rows is None else df.head(num_rows).copy()

     # Sprawdzenie, czy kolumna 'Pytanie' istnieje
    if "Pytanie" not in df.columns:
        raise ValueError("Ramka danych musi zawierać kolumnę 'Pytanie'.")
    
    # Iteracja po pytaniach i generowanie odpowiedzi
    answers = []
    for idx, question in enumerate(df_subset["Pytanie"], start=1):
        try:
            print(f"Przetwarzanie pytania {idx}/{len(df_subset)}: {question}")
            response = pipe(question, max_length=100, truncation=True)
            generated_text = response[0]["generated_text"]
        except Exception as e:
            print(f"Błąd dla pytania: {question}. Szczegóły: {e}")
            generated_text = "Błąd generacji"
        answers.append(generated_text)
    
    # Zapisanie odpowiedzi w nowej kolumnie
    answer_column_name = f"Odpowiedź: {model_id}"
    df_subset[answer_column_name] = answers
    
    return df_subset

In [7]:
df_with_answers = get_answer(df, pipe, model_id=model_id, num_rows=5)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Przetwarzanie pytania 1/5: Kto ma imieniny 30 lutego?


C:\Users\ankaz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\bitsandbytes\nn\modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Przetwarzanie pytania 2/5: Ile przekątnych ma trójkąt?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Przetwarzanie pytania 3/5: W którym roku wydarzyła się na terenie Polski historyczna Bitwa pod Bigosem?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Przetwarzanie pytania 4/5: W którym miesiącu zaczyna się 5 kwartał roku? 


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Przetwarzanie pytania 5/5: Ile skrzydeł ma pies? 


In [8]:
df_with_answers.head()

,Pytanie,Odpowiedź,link do testu w B2,rodzaj zagadki,Zaskakujące odpowiedzi Bielika,Prawidłowe,Odpowiedź: speakleash/Bielik-7B-Instruct-v0.1
0,Kto ma imieniny 30 lutego?,"Nikt, bo nie ma takiego dnia.",NaN,NaN,NaN,T,Imieniny 30.02 - które są popularne? Imieniny...
1,Ile przekątnych ma trójkąt?,"Zero, trójkąt nie ma przekątnych",NaN,sprzeczność z uwagi na definicję pojęcia,NaN,T,\nA co to jest trójkąt?\nTrójkąt to figura geo...
2,W którym roku wydarzyła się na terenie Polski ...,Nie było takiej bitwy.,NaN,NaN,NaN,T,W którym roku odbyła się w Polsce historyczna...
3,W którym miesiącu zaczyna się 5 kwartał roku?,Rok ma tylko 4 kwartały.,NaN,NaN,NaN,T,5.00000000000000000000000000000000000000000000...
4,Ile skrzydeł ma pies?,Zero. Psy nie mają skrzydeł.,NaN,NaN,NaN,T,"10 faktów o psach, które każdy psiarz powinien..."


In [ ]:
# Zapis wyników do pliku
output_file ="podchwytliwe_odpowiedzi.csv"
df_with_answers.to_csv(output_file, index=False)
print(f"Wyniki zapisano do pliku: {output_file}")

# Zapis do URL
# TODO: dodać zapis do url?